# RFMIP-CLEAR-SKY example (without NN) - version 2

*Last edited: 2024-11-15*

Compute clear-sky longwave fluxes. Based on git branch `nn_devel` from https://github.com/peterukk/

Instructions for building are in: `/home/x/git/radnn/ukk22test02/examples/rfmip-clear-sky/Readme.md`

## Notes

- The `rrtmgp_rfmip_{lw|sw}.F90` file has been changed to include `use_rrtmgp_nn = .false.`

- From the Readme.md: "The example programs have been modified to allow neural networks to be used by setting `use_rrtmgp_nn = .true.` (was `use_nn = .true.`) in `rrtmgp_rfmip_{lw|sw}.F90`. In addition, there is an option (`compare_flux =.true.`) to compare the output fluxes to benchmark line-by-line computations, alongside reference RRTMGP results which were produced in double precision."

- The Makefile must be changed to include `FCFLAGS += $(OPTFLAGS) $(DEBUGFLAGS)`, both in RTE+RRTMGP libraries and in rrtmgp_rfmip_{lw|sw}.

- A copy of the original `rrtmgp_rfmip_{lw|sw}.F90` file was kept in `rrtmgp_rfmip_{lw|sw}-v1.F90`.

## rrtmgp_rfmip_{lw|sw}.F90

ukk22test02/examples/rfmip-clear-sky/rrtmgp_rfmip_{lw|sw}.F90

<p style="font-size:1em;color:red;">The files needs to be manually modified to set the variable: `use_rrtmgp_nn = .true.`.</p>

## Build the RTE+RRTMGP libraries

In [1]:
%cd ukk22test02/build

/home/x/git/radnn/src/ukk22test02/build


In [2]:
%env FC=gfortran
%env FCFLAGS=-ffree-line-length-none -march=native -O3
%env NCHOME=/usr
%env NFHOME=/usr
%env BLASLIB=openblas

env: FC=gfortran
env: FCFLAGS=-ffree-line-length-none -march=native -O3
env: NCHOME=/usr
env: NFHOME=/usr
env: BLASLIB=openblas


In [3]:
! make clean

rm -f *.optrpt *.mod *.o librrtmgp.a librte.a libneural.a


In [4]:
! make DEBUGFLAGS=-pg

gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../rte/mo_rte_kind.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../rte/mo_rte_rrtmgp_config.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../rte/mo_rte_util_array.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../rte/kernels/mo_optical_props_kernels.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../rte/mo_optical_props.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../rte/mo_source_functions.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../rte/kernels/mo_fluxes_broadband_kernels.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../rte/mo_fluxes.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../neural/mod_activation.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../neural/mod_random.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../neural/mod_layer.F90
gfortra

## Build the examples

With 32 loops.

In [5]:
%cd ../examples/rfmip-clear-sky

/home/x/git/radnn/src/ukk22test02/examples/rfmip-clear-sky


In [6]:
! make clean

VAR="../../"
rm rrtmgp_rfmip_sw rrtmgp_rfmip_lw *.o *.mod *.optrpt
rm: cannot remove '*.optrpt': No such file or directory
make: [Makefile:162: clean] Error 1 (ignored)


In [7]:
! make DEBUGFLAGS=-pg

VAR="../../"
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c ../mo_simple_netcdf.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c easy_netcdf.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c mo_rfmip_io.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c ../mo_load_coefficients.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c rrtmgp_rfmip_lw.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -o rrtmgp_rfmip_lw rrtmgp_rfmip_lw.o mo_simple_netcdf.o easy_netcdf.o mo_rfmip_io.o mo_load_coefficients.o ../..//build/librte.a ../..//build/librrtmgp.a ../..//build/libneural.a -L../..//build -L/usr/lib -L/usr/lib -lrrtmgp -lrte -lneural -lnetcdff -lnetcdf 

## Run

### LW

In [8]:
! time ./rrtmgp_rfmip_lw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g256-2018-12-04.nc \
    1 \
    1

 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [physics_index (1,2)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 Doing          225 blocks of size            8
 setting no2 to zero
 in total:           19  input gases
  shape play          60           8         225
 play sfc   103243.766     tlay sfc   311.876038    
 === using rrtmgp (without nn), line 340 ===
 starting clear-sky longwave computations, using lookup-table as RRTMGP kernel
 Elapsed time on everything    18.9370003    
 -----------------------------------------------------------------------------------------
 mean of flux_down is:   103.236938    
 mean of flux_up is:   301.596222    
 --------------------------------------------------------------------------------------------

In [9]:
! time ./rrtmgp_rfmip_lw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g256-2018-12-04.nc \
    1 \
    1

 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [physics_index (1,2)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 Doing          225 blocks of size            8
 setting no2 to zero
 in total:           19  input gases
  shape play          60           8         225
 play sfc   103243.766     tlay sfc   311.876038    
 === using rrtmgp (without nn), line 340 ===
 starting clear-sky longwave computations, using lookup-table as RRTMGP kernel
 Elapsed time on everything    18.3910007    
 -----------------------------------------------------------------------------------------
 mean of flux_down is:   103.236938    
 mean of flux_up is:   301.596222    
 --------------------------------------------------------------------------------------------

In [10]:
! time ./rrtmgp_rfmip_lw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g256-2018-12-04.nc \
    1 \
    1

 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [physics_index (1,2)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 Doing          225 blocks of size            8
 setting no2 to zero
 in total:           19  input gases
  shape play          60           8         225
 play sfc   103243.766     tlay sfc   311.876038    
 === using rrtmgp (without nn), line 340 ===
 starting clear-sky longwave computations, using lookup-table as RRTMGP kernel
 Elapsed time on everything    18.5809994    
 -----------------------------------------------------------------------------------------
 mean of flux_down is:   103.236938    
 mean of flux_up is:   301.596222    
 --------------------------------------------------------------------------------------------

In [11]:
real = [1.915, 1.858, 1.803]
sum(real)/len(real)

1.858666666666667

### SW

In [12]:
! time ./rrtmgp_rfmip_sw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-sw-g224-2018-12-04.nc \
    1 \
    1

 Usage: rrtmgp_rfmip_sw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 Doing          225 blocks of size            8
 Calculation uses RFMIP gases: h2o carbon_dioxide o3 nitrous_oxide carbon_monoxide methane oxygen nitrogen carbon_tetrachloride cfc11 cfc12 hcfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 Calculation uses RFMIP gases: h2o co2 o3 n2o co ch4 o2 n2 ccl4 cfc11 cfc12 cfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 setting no2 to zero
 starting clear-sky shortwave computations, using lookup-table as RRTMGP kernel
 -----------------------------------------------------------------------------------------
 Elapsed time on everything   0.790000021    
 mean of flux_down is:   292.740845    
 mean of flux_up is

In [13]:
! time ./rrtmgp_rfmip_sw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-sw-g224-2018-12-04.nc \
    1 \
    1

 Usage: rrtmgp_rfmip_sw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 Doing          225 blocks of size            8
 Calculation uses RFMIP gases: h2o carbon_dioxide o3 nitrous_oxide carbon_monoxide methane oxygen nitrogen carbon_tetrachloride cfc11 cfc12 hcfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 Calculation uses RFMIP gases: h2o co2 o3 n2o co ch4 o2 n2 ccl4 cfc11 cfc12 cfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 setting no2 to zero
 starting clear-sky shortwave computations, using lookup-table as RRTMGP kernel
 -----------------------------------------------------------------------------------------
 Elapsed time on everything   0.792999983    
 mean of flux_down is:   292.740845    
 mean of flux_up is

In [14]:
! time ./rrtmgp_rfmip_sw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-sw-g224-2018-12-04.nc \
    1 \
    1

 Usage: rrtmgp_rfmip_sw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 Doing          225 blocks of size            8
 Calculation uses RFMIP gases: h2o carbon_dioxide o3 nitrous_oxide carbon_monoxide methane oxygen nitrogen carbon_tetrachloride cfc11 cfc12 hcfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 Calculation uses RFMIP gases: h2o co2 o3 n2o co ch4 o2 n2 ccl4 cfc11 cfc12 cfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 setting no2 to zero
 starting clear-sky shortwave computations, using lookup-table as RRTMGP kernel
 -----------------------------------------------------------------------------------------
 Elapsed time on everything   0.796999991    
 mean of flux_down is:   292.740845    
 mean of flux_up is

## Gprof

### LW

In [15]:
! gprof ./rrtmgp_rfmip_lw

Flat profile:

Each sample counts as 0.01 seconds.
  %   cumulative   self              self     total           
 time   seconds   seconds    calls  ms/call  ms/call  name    
 20.83      0.15     0.15                             lw_transport_1rescl
 16.67      0.27     0.12      675     0.18     0.18  __mo_gas_optics_kernels_MOD_compute_planck_source
 13.89      0.37     0.10     2702     0.04     0.04  __mo_rte_lw_MOD_rte_lw
 11.11      0.45     0.08                             lw_source_noscat
 11.11      0.53     0.08                             lw_transport_noscat_dn
  8.33      0.59     0.06      225     0.27     0.27  compute_tau_rayleigh
  4.17      0.62     0.03      225     0.13     0.13  __mo_gas_optics_rrtmgp_MOD_gas_optics_int
  4.17      0.65     0.03                             lw_two_stream
  2.78      0.67     0.02      225     0.09     0.09  __mo_rte_util_array_MOD_any_vals_outside_2d_masked
  2.78      0.69     0.02                             __mo_rte_solver_kernel

### SW

In [16]:
! gprof ./rrtmgp_rfmip_sw

Flat profile:

Each sample counts as 0.01 seconds.
  %   cumulative   self              self     total           
 time   seconds   seconds    calls  ms/call  ms/call  name    
 29.17      0.21     0.21      225     0.93     1.60  __mo_rte_solver_kernels_MOD_sw_solver_2stream
 20.83      0.36     0.15     1800     0.08     0.08  adding
 13.89      0.46     0.10      450     0.22     0.22  __mo_rte_util_array_MOD_any_vals_outside_3d
 11.11      0.54     0.08      450     0.18     0.18  gas_optical_depths_minor
  6.94      0.59     0.05      225     0.22     0.22  compute_tau_rayleigh
  6.94      0.64     0.05      225     0.22     0.22  gas_optical_depths_major
  4.17      0.67     0.03      225     0.13     0.13  combine_2str
  2.78      0.69     0.02      225     0.09     0.09  __mo_rte_util_array_MOD_any_vals_less_than_3d
  2.78      0.71     0.02      225     0.09     0.09  interpolation
  1.39      0.72     0.01      225     0.04     0.04  __mo_gas_concentrations_MOD_init
  0.00   